In [27]:
pip install tflearn


Note: you may need to restart the kernel to use updated packages.


In [24]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
import numpy
import tflearn
import tensorflow
import random
import json

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Murugan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [25]:
with open("intents.json")as file:
    data=json.load(file)
    
print(data)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day', 'Whats up'], 'responses': ['Hello!', 'Good to see you again!', 'Hi there, how can I help?'], 'context_set': ''}, {'tag': 'goodbye', 'patterns': ['cya', 'See you later', 'Goodbye', 'I am Leaving', 'Have a Good day'], 'responses': ['Sad to see you go :(', 'Talk to you later', 'Goodbye!'], 'context_set': ''}, {'tag': 'java', 'patterns': ['What is JAVA', 'why java', 'need for java', 'Java', 'java?'], 'responses': ['Java is a high-level programming language', 'Java is a collection of objects. It was developed by Sun Microsystems.', 'platform independend'], 'context_set': ''}, {'tag': 'Constructor', 'patterns': ['What do you mean by Constructor?', 'what is constructor', 'why constructor?'], 'responses': ['When a new object is created in a program a constructor gets invoked corresponding to the class.', 'The constructor is a method which has the same name as the class name.', 'If a user

In [26]:
words=[]
labels=[]
docs_x=[]
docs_y=[]

for intent in data["intents"]:
    for pattern in intent["patterns"]:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])
    if intent["tag"] not in labels:
        labels.append(intent["tag"])
             
        

['Constructor', 'JVM', 'abstract', 'boolean', 'break', 'byte', 'catch', 'continue', 'final', 'finally', 'goodbye', 'greeting', 'heap', 'java', 'static', 'super']


In [27]:
words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)
training = []
output = []
out_empty = [0 for _ in range(len(labels))]
for x,doc in enumerate(docs_x):
    bag=[]
    wrds=[stemmer.stem(w) for w in doc]
    
    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)
            
    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])]=1
    
    training.append(bag)
    output.append(output_row)
    
training = numpy.array(training)
output = numpy.array(output)
            

In [20]:
print(output)

[[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 

In [28]:
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tensorflow.python.framework import ops
ops.reset_default_graph()

net = tflearn.input_data(shape=[None,len(training[0])])
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,len(output[8]),activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

model.fit(training,output,n_epoch=1000,batch_size=8,show_metric=True)
model.save("model.tflearn")

Training Step: 6999  | total loss: 0.53621 | time: 0.025s
| Adam | epoch: 1000 | loss: 0.53621 - acc: 0.9445 -- iter: 48/49
Training Step: 7000  | total loss: 0.48309 | time: 0.030s
| Adam | epoch: 1000 | loss: 0.48309 - acc: 0.9501 -- iter: 49/49
--
INFO:tensorflow:C:\Users\Murugan\chatbot\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [21]:
print(len(output[8]))

16


In [29]:
def bag_of_words(s,words):
    bag=[0 for _ in range(len(words))]
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]
    
    for se in s_words:
        for i,w, in enumerate(words):
            if w == se:
                bag[i]=1
                
    return numpy.array(bag)

In [30]:
def chat():
    print("start talking with the bot (type quit to stop)")
    while True:
        inp=input("you: ")
        if inp.lower() =="quit":
            break
            
        results = model.predict([bag_of_words(inp,words)])
        results_index = numpy.argmax(results)
        tag = labels[results_index]
        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']
                
        print(random.choice(responses))
        

In [ ]:
chat()

start talking with the bot (type quit to stop)


you:  hi


Good to see you again!


you:  JVM


JVM (Java Virtual Machine) is an abstract machine.


...

When a new object is created in a program a constructor gets invoked corresponding to the class.


you:  super


Java super keyword is a reference variable that is used to refer parent class object. It can be used to invoke immediate parent class method.


you:  goodbye


Talk to you later


In [ ]:
Goodbye
